In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt

In [ ]:
labels2id = {
    'scab': 0,
    'healthy': 1,
    'frog_eye_leaf_spot': 2,
    'rust': 3,
    'complex': 4,
    'powdery_mildew': 5
}

id2labels = {v:k for k,v in labels2id.items()}

label_classes = labels2id.keys()

In [ ]:
image_size = [600,600]
channels = 3
num_classes = len(label_classes)

In [ ]:
model = keras.models.load_model('../input/plant-pathology-2021/ResNet152V2-600.h5')

In [ ]:
def load_predict_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image, channels=channels)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, image_size)
    image = tf.expand_dims(image,axis=0)
    return image

In [ ]:
test_path = '../input/plant-pathology-2021-fgvc8/test_images/'

submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
for row in submission.index:
    
    image = load_predict_image(test_path+submission.loc[row,'image'])
    predict = model.predict(image)[0]
    predict = [1 if i>0.5 else 0 for i in predict]
    result = []
    for i,j in enumerate(predict):
        if j:
            result.append(id2labels.get(i))
    result = ' '.join(result)
    submission.loc[row,'labels'] = result

In [ ]:
submission.to_csv('submission.csv',index=False)